In [13]:
using JuMP
import LinearAlgebra
import SCS

In [14]:
N, d = 2, 2

(2, 2)

In [15]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

ρ = [random_state(d) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}}:
 [0.7137236111327925 + 0.0im -0.1299712100438686 + 0.14187194750338206im; -0.1299712100438686 - 0.14187194750338206im 0.2862763888672076 + 0.0im]
 [0.2080932422704056 + 0.0im 0.2617946248567618 - 0.18260830915594517im; 0.2617946248567618 + 0.18260830915594517im 0.7919067577295944 + 0.0im]

In [16]:
model = Model(SCS.Optimizer)
set_silent(model)

In [17]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]

In [18]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] - 1                  _[2] + _[6] + _[4] im + _[8] im
 _[2] + _[6] - _[4] im - _[8] im  _[3] + _[7] - 1] ∈ Zeros()

In [19]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] - 1                  _[2] + _[6] + _[4] im + _[8] im
 _[2] + _[6] - _[4] im - _[8] im  _[3] + _[7] - 1] ∈ Zeros()

In [20]:
@objective(
    model,
    Max,
    sum(real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.35686180556639624 _[1] - 0.1299712100438686 _[2] + 0.14187194750338206 _[4] + 0.1431381944336038 _[3] + 0.1040466211352028 _[5] + 0.2617946248567618 _[6] - 0.18260830915594517 _[8] + 0.3959533788647972 _[7]

In [21]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.58617e-01
  Dual objective value : 8.58618e-01

* Work counters
  Solve time (sec)   : 1.02814e-01


In [22]:
objective_value(model)

0.858617382249673

In [23]:
0.5 + 0.25 * sum(LinearAlgebra.svdvals(ρ[1] - ρ[2]))

0.85861887020659

In [24]:
solution = [value.(e) for e in E]

2-element Vector{Matrix{ComplexF64}}:
 [0.8524830349008435 + 0.0im -0.2731062441756322 + 0.22620038888593266im; -0.2731062441756322 - 0.22620038888593266im 0.147516962420438 + 0.0im]
 [0.1475169623011509 + 0.0im 0.2731062441889881 - 0.22620038888999766im; 0.2731062441889881 + 0.22620038888999766im 0.8524830347926187 + 0.0im]